In [1]:
%run ./import_apple_pie_loc.ipynb
#exp = ap.Exper(pjoin(rh30_path, rh30_dirs[5]))
exp = ap.Exper(pjoin(rhabdo_root_path_local, e18_11_09))

reading in wells : 0.84 seconds
all exper init : 1.63 seconds


In [2]:
import ipywidgets 
import matplotlib.pyplot as plt

In [3]:
def col_dict_to_cols(col_dict) :
    cols = []
    for key in col_dict.keys() :
        cols.append(col_dict[key]) 
    return cols

def col_dict_to_rows(col_dict) :
    cols = []
    for key in col_dict.keys() :
        cols.append(col_dict[key]) 
    rows = ap.ap_utils.rotate(cols)
    return rows

In [4]:
class group :
    def __init__(self, exper, name) :
        self.exper = exper
        self.name = name
        
        self.condits = {}
        
        for condit_tup in self.exper.groups[0][name] :
            self.condits[condit_tup] = self.exper.condits[condit_tup]
    
        self.ctkeys = list(self.condits.keys())
        self.cskeys = []
        self.ckeys_dict = {}
        
        
        
        for t in self.ctkeys :
            self.cskeys.append(str(t))
            self.ckeys_dict[str(t)] = t
    
    def __str__(self) :
        return self.name
    
    ## bad practice
    def __repr__(self) :
        return str(self)

In [5]:
groups = exp.groups[0]

In [6]:
def plot_condit(condit_skey) :
    for gr in groups.keys() :
        g = group(exp,gr) 
        try :
            condit = g.condits[g.ckeys_dict[condit_skey]]
            break
        except :
            pass
    plt.plot(condit.t_int,col_dict_to_rows(condit.dists))
    
def scatter_condit(condit_skey) :
    for gr in groups.keys() :
        g = group(exp,gr) 
        try :
            condit = g.condits[g.ckeys_dict[condit_skey]]
            break
        except :
            pass
    for col in condit.dists.values() :
        plt.scatter(condit.t_int,col)

def condit_meds(condit_skey) :
    for gr in groups.keys() :
        g = group(exp,gr) 
        try :
            condit = g.condits[g.ckeys_dict[condit_skey]]
            break
        except :
            pass
    condit.make_cen_tens()
    plt.scatter(condit.t_int,condit.dist_meds)
    ax = plt.gca()
    ax.set_ylim((0,20))


ffs = []
i = 0
for gg in groups.keys() :
    g = group(exp,gg)
    select_condit = ipywidgets.Select(
        options = g.cskeys,
        value = (g.cskeys[0]),
        description='condition'
    )

    tabs = ipywidgets.Tab() 
    i += 1
    f = ipywidgets.interactive(condit_meds, condit_skey=select_condit)
    
    ffs.append(ipywidgets.HBox(children=f.children))
tabs.children = ffs

i = 0
for gg in groups.keys() :
    g = group(exp,gg)
    tabs.set_title(i,g.name)
    i+=1
display(tabs)

select_condit.value = (g.cskeys[1])
